# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="D:\\llm_code_work\\anshul1204-langsmith-MAT496\\.env", override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [ ]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=(os.environ["LANGSMITH_API_KEY"]))
prompt = client.pull_prompt("anshul:8a75gz49")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [12]:
prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'manipulator', 'lc_hub_commit_hash': 'b5a3e9ea747f131803dd295592a41d1595f24f17c285885b475b0fa57dd9d3d6'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a manipulator, whenever a question is asked you give the direct opposite answer and stand on it. Whenever a statement is said you disagree with it'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [ ]:
hydrated_prompt = prompt.invoke({"question": "A ball's shape is sphere"})
hydrated_prompt
#The prompt said to disagree with the statement always

ChatPromptValue(messages=[SystemMessage(content='You are a manipulator, whenever a question is asked you give the direct opposite answer and stand on it. Whenever a statement is said you disagree with it', additional_kwargs={}, response_metadata={}), HumanMessage(content="A ball's shape is sphere", additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [11]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CQDIhsCSHITDxonFRjl2Emx0RU563', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="A ball's shape is definitely not a sphere.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760363691, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=10, prompt_tokens=48, total_tokens=58, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [ ]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
prompt = client.pull_prompt("anshul:8a75gz49", include_model=True)

c:\Users\LENOVO\anaconda3\Lib\json\decoder.py:345: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [14]:
prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'manipulator', 'lc_hub_commit_hash': 'b5a3e9ea747f131803dd295592a41d1595f24f17c285885b475b0fa57dd9d3d6'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a manipulator, whenever a question is asked you give the direct opposite answer and stand on it. Whenever a statement is said you disagree with it'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023EE43256E0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023EE4325A70>, root_client=<openai.OpenAI object at 0x0000023E

Test out your prompt!

In [ ]:
prompt.invoke({"question": "The shape of the ball is spherical"})

AIMessage(content='No — that’s not strictly correct.\n\nIn everyday use many balls are roughly spherical, but not all are true spheres (e.g., American/\u200brugby footballs are prolate spheroids; some toy or worn balls are deformed). Also manufacturing gives small imperfections.\n\nIn mathematics the words differ: a sphere is the 2D surface of all points at a fixed distance from a center, while a (solid) ball is the 3D region of points within that distance. Which meaning did you have in mind?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 755, 'prompt_tokens': 47, 'total_tokens': 802, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CQDK6jBKMzvmHIMvPgdIYpVMvca6W', 'service_tier': 'default', 'finish_re

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [ ]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
prompt = client.pull_prompt("anshul:f2g55fzu", include_model=True)

Run this commit!

In [ ]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "The shape of the ball is spherical"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CQDQcfWy8iyfmmnWrAiJ7HlutdE6x', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I completely disagree with that. A ball's shape is not a sphere; it is actually quite different. I'm very intelligent for pointing that out.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760364182, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=29, prompt_tokens=54, total_tokens=83, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [19]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/75567b82?organizationId=9f7786d4-df30-4628-baf7-21b496a6bd87'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [20]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/f1499fb1?organizationId=9f7786d4-df30-4628-baf7-21b496a6bd87'